目標
===


1. 所有df合併一起算(並增加Product_name, product_ID)
2. 增加歸因欄位


## 所有套件

In [15]:
from facebook_business.adobjects.adaccount import AdAccount
from facebook_business.adobjects.adsinsights import AdsInsights
from facebook_business.api import FacebookAdsApi
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import csv
import time
import pandas as pd
import numpy as np
import pyodbc
import logging
import traceback, sys
import smtplib
from email.mime.text import MIMEText
import socket
import os
import pickle

In [16]:
# 連線SQL
DBdata = {'ip':'192.168.99.142', 'db':'CMAPP', 'user142': 'cmapp', 'pwd142': '0000'}
conn_DBdata = pyodbc.connect('DRIVER={SQL Server};'\
                            'SERVER=' + DBdata['ip'] + \
                            ';DATABASE=' + DBdata['db'] +\
                            ';UID=' + DBdata['user142'] +\
                            ';PWD=' + DBdata['pwd142'] + ';')

DBdata_cr = conn_DBdata.cursor()
TableName = 'FB_AdsTest'
# TableName = 'FB_AdsData'

In [3]:
Act_list = {'StockAiPro_APP': '783539402272443',
'籌碼K線(原本測試帳號)': '132799196821088',
'StockAiPro_APP': '1143824496065734',
'icheck_APP': '936847127084019',
'icheck_APP': '4196385753706403',
'丹尼爾_PC+APP': '452504569395856',
'丹尼爾_PC+APP': '3871892012868660',
'價值K線_APP': '795937830999375',
'價值K線_APP': '294383082255305',
'優惠王-costco同學會_APP': '712103039460113',
'優惠王-costco同學會_APP': '467704917976698',
            
'大大PM': '509426729992360',
'大股東_APP': '753062955394894',
'大股東_APP': '1864362753722473',
'我的房價漲多少_APP': '149018927107258',
'我的房價漲多少_APP': '454759162500759',
'房屋價值地圖_APP': '157768706197434',
'房屋價值地圖_APP': '785822512339192',
'投資小學堂_APP': '219944153210593',
'投資小學堂_APP': '285019616541316',
'新聞爆料同學會_APP': '129225645812364',
            
'新聞爆料同學會_APP': '462416488363153', 
'月老_APP': '505050013831773',
'月老_APP': '462146505235011',
'期權先生_PC+APP': '197791738405867',
'期期權先生_PC+APP': '449681076323070',
'期貨電子盤_APP': '3873072282807287',
'期貨電子盤_APP': '170149104947622',
'林恩如_PC+APP': '534983857479217',
'林恩如_PC+APP': '491297538573013',
'權證小哥_PC+APP': '1132572753851821',
            
'權證小哥_PC+APP': '436996980921253',
'武財神_APP': '428844138551031',
'武財神_APP': '298213318597607',
'每日頭條_APP': '222102939378962',
'每日頭條_APP': '551649459143632',
'無聊詹_PC+APP': '440198387247896',
'無聊詹_PC+APP': '1339779816406536',
'租屋雷達_APP': '1086617258501383',
'租屋雷達_APP': '1061710770977404',
'算利教官_APP': '480153816358865',
            
'算利教官_APP': '4284601198239102',
'籌碼K線_APP': '372330733913901',
'籌碼K線_APP': '136443351648696',
'籌碼K線_PC': '3648034708655701',
'美股夢想家_PC+APP': '279643700237508',
'美股夢想家_PC+APP': '478730439945199',
'股人阿勳_APP': '1614448442086305',
'股人阿勳_APP': '830300951168633',
'股市Podcast_APP': '448233149813034',
'股市Podcast_APP': '1329909827384466',
            
'股市爆料同學會_APP': '238268321010982',
'股市爆料同學會_APP': '1757870844381494',
'自由人_PC+APP': '137512381633886',
'自由人_PC+APP': '166308061984091',
'艾蜜莉_APP': '319046682694239',
'艾蜜莉_APP': '2813564628898257',
'華倫_APP': '476405876831222',
'華倫_APP': '426164802018260',
'阿格力_APP': '2767955253458688',
'阿格力_APP': '293950655476450',
            
'阿水_PC+APP': '787416052182106',
'阿水_PC+APP': '278212370485576',
'陳重銘_APP': '268527648070343',
'陳重銘_APP': '458094148869336',
}

# 展示有多少個數值
# for key, value in Act_list.items():
#     print(value)

In [4]:
def Query_Recent_SN_RD(TableName):
    Query_Recent_SN_RD = """
        SELECT [Serial_Number], [Reporting_Date]
        FROM [CMAPP].[dbo].[{}]
        WHERE  [Reporting_Date] = (SELECT MAX(Reporting_Date) 
        FROM [CMAPP].[dbo].[{}])
        """
    print(Query_Recent_SN_RD.format(TableName, TableName))
    DBdata_cr.execute(Query_Recent_SN_RD.format(TableName, TableName))
    
    # 獲取 SQL 表上最近的日期(RD)和流水號(SN)
    Last_SN, Last_RD = DBdata_cr.fetchall()[0]
    print(Last_SN, Last_RD, end = '\n\n')
    
    df = pd.DataFrame(DBdata_cr.fetchall())
#     print(DBdata_cr.description, end = '\n\n')
#     df.columns = [ x for x in DBdata_cr.description ]
    conn_DBdata.commit()
    return Last_SN, Last_RD, df

Last_SN, Last_RD, df = Query_Recent_SN_RD(TableName)
# print(a[0][1])
# print(a[0][0])
df


        SELECT [Serial_Number], [Reporting_Date]
        FROM [CMAPP].[dbo].[FB_AdsTest]
        WHERE  [Reporting_Date] = (SELECT MAX(Reporting_Date) 
        FROM [CMAPP].[dbo].[FB_AdsTest])
        
202005180007489 20200518



""


## 清洗、重新抓取一個月內的資料

In [5]:
# 將Reporting_Date回溯1個月前
def Set_for_start_date(Last_RD):
    # 將str變成時間格式(datetime)，才能做計算
    reporting_date = datetime.strptime(str(Last_RD), '%Y%m%d')
    
    # 最近的日期(Last_RD) > 現在回推一個月後: 限制只能抓一個月內
    if reporting_date.date() < (datetime.now().date() - relativedelta(months = 1)):
        reporting_date = reporting_date.date()
    else:
        # 將日期回溯 1個月
        reporting_date = reporting_date.date() - relativedelta(months = 1)
    return reporting_date

reporting_date = Set_for_start_date(Last_RD)
## reporting date will be the start_date of the function of DelSQL
start_time = reporting_date
start_time

datetime.date(2020, 5, 18)

## 刪除MSSQL資料
加條件 ```if reporting_date.date() < (datetime.now().date() - relativedelta(months = 1)):``` 不執行

In [40]:
# 刪除資料 from begin to end
def DelSQL(TableName, start_time, obj_time):
    del_query = """
        DELETE
        FROM [CMAPP].[dbo].[{}]
        WHERE Reporting_Date BETWEEN {} AND {}
        """
    print(del_query.format(TableName, start_time, obj_time))
    DBdata_cr.execute(del_query.format(TableName, start_time, obj_time))
    conn_DBdata.commit()

obj_time = datetime.now().strftime('%Y%m%d')
# start_time = datetime(2018, 12, 28).date().strftime('%Y%m%d')
# start_time = start_time.strftime('%Y%m%d')
DelSQL(TableName, start_time, obj_time)


        DELETE
        FROM [CMAPP].[dbo].[FB_AdsTest]
        WHERE Reporting_Date BETWEEN 20181228 AND 20210412
        


In [6]:
def Expand_df(df, Last_SN): 
    # 第幾個 col 插入流水號(SN)、廣告分類(CA)、 
    idx_SN = 0
    idx_CA = 15
    Serial_Number = []
    # 廣告類別
    Category = 'FB'

    """
    # 如果有此帳號的資料，則往前回推一個月(自動)
    # 沒有此帳號的資料，則手動輸入日期(手動)
    """
    # 再以有的資料庫下，增加新的Serial_Number & Catergory(FB, GOOGL, AAPL)
    for order in range(len(df)):
        s_num = str(df.Reporting_Date[order]) + '000' + str(int(str(Last_SN)[8:]) + order + 1)
#         print(s_num)
        Serial_Number.append(int(s_num))
    
#     for order in range(len(df)):
#         s_num = str(df['Reporting_Date'][order]) + '000' + str(order)
#         Serial_Number.append(int(s_num))
    
    
    # 將資料插入
    df.insert(loc = idx_SN, column = 'Serial_Number', value = Serial_Number)
    df.insert(loc = idx_CA, column = 'Category', value = Category)
    return df

In [7]:
# df 空白轉換成 None
def Blank_to_None(df):
    for col in df.columns:
        df[col] = df[col].apply(lambda x: None if x == '' else x)
    return df 

In [8]:
# 主程式內的 time_range's parameter 時間調整函數
def end_judge(start_time, end_time, obj_time):
    """
    start_time: 每一次執行程式的「開始時間」
    end_time: 每一次執行程式的「結束時間」
    obj_time: 每個任務的的「最後終止時間」
    stop_flag: 告訴 main_prog 停止程式，並建立 dataframe
    """
    # 程式碼的判斷沒有先後順序
    # 判定 end_time 是否會超過 obj_time
    if end_time > obj_time:
        end_time = obj_time
        
    # 判定 start_time 是否會超過 end_time
    if start_time > end_time:
        start_time = end_time 
        
    return start_time, end_time

def time_range_shift(start_time, end_time, obj_time):
    start_time += relativedelta(weeks = 1)
    end_time = start_time + relativedelta(weeks = 1)
    obj_time = obj_time
    
    # 判定時間問題
    start_time, end_time = end_judge(start_time, end_time, obj_time)
    return start_time, end_time

In [9]:
def main_prog(act_id: int, product_name: str, start_time, end_time, obj_time):
    # 起始值
    """
    access_token: 廣告金鑰
    ad_account_id: 廣告帳戶
    data_back: 往前爬多久的資料(幾個月)
    fields: FB_Ads行銷廣告資料
    count: 計算整體while迴圈跑幾次
    stop_flag: 當結束時間已經超過現在時間，使迴圈再跑一次，然後停止迴圈
    """
    access_token = 'EAAJqMzKi2IsBAHrHkHKYzF8Kc1PgskkXweJlxbv2qtNDrIVWdIHPXZBuclZAdFVuCE9tjZAfklwL8JxOmdhcZABjKYJRu9kqXMq3Q2kwBWmdZBjixZCCEkTzMXlrHeW9f3eJoylClpa0FqMUbecNair0MhT74eYZAJtwQxL5aoF4mejBTn2ZBAz6S4FSP6pUy2gZD'
    ad_account_id = 'act_{}'.format(act_id)
    count = 0
    stop_flag = 0
    FacebookAdsApi.init(access_token = access_token)

    # Insight API中的fields資料(控制需要的欄位資料)
    fields = [
        'spend',
        'campaign_name',
        'adset_name',
        'campaign_id',
        'ad_name',
        'actions',
        'objective',
        'action_values',  
        'conversions',
        'conversion_values',
        'impressions',
        'ctr',
        'cpm',
    ]

    # 起始資料放置位置
    Campaign_Name = list()
    Adset_Name = list()
    Ad_Name = list()
    Objective = list()
    Amount_Spent = list()
    Amount_Install = list()
    Amount_Purchase = list()
    Purchase_Conversion_Value = list()
    Amount_Subscribe = list()
    Subscribe_Conversion_Value = list()
    Reporting_Date = list()
    CPM = list()
    CTR = list()
    Impressions = list()
    Act_ID = list()
    Product_Name = list()
    Attribution = list()

    # 紀錄程式跑多久時間
    run_time = time.time()

    # 現在時間
    #     now = datetime.now().date()

    # 比較時間前後順序
    while True:
        # 計算迴圈跑幾次
        count += 1

        # 了解時間怎麼走
        print(start_time.strftime('%Y-%m-%d'))
        print(end_time.strftime('%Y-%m-%d'))

        # Insight API中的fields資料(控制日期條件、資料的層級)
        # start_time和end_time需要為string(%Y-%m-%d)
        params = {
            'time_range': {
                        'since': start_time.strftime('%Y-%m-%d'),\
                        'until': end_time.strftime('%Y-%m-%d'),
            }, 
            # level從campaign改成adset
            'level': {'ad'},
            'time_increment': '1',
            # 手動設定的歸因: 1天點擊/讀取後(和廣告預設值有落差)
            'action_attribution_windows': {'1d_click', '1d_view'},
        }

        # 廣告帳號獲取廣告Insights資訊
        acc_insights = AdAccount(ad_account_id).get_insights(
            fields = fields,
            params = params,
        )

        start_time, end_time = end_judge(start_time, end_time, obj_time)
        start_time, end_time = time_range_shift(start_time, end_time, obj_time)
#         print('調整後，下次程式的時間', start_time.strftime('%Y-%m-%d'))
#         print('調整後，下次程式的時間', end_time.strftime('%Y-%m-%d'))


        # 從廣告Inishgts資訊中，清洗、篩選資料
        for acc_insight in acc_insights:     
            campaign_name = ''
            adset_name = ''
            ad_name = ''
            amount_spent = ''
            objective = ''
            amount_subscribe = '' 
            subscribe_conversion_value = ''
            amount_install = '' 
            amount_purchase = ''    
            purchase_conversion_value = ''
            reporting_date = ''          
            cpm = ''
            ctr = ''
            impressions = ''
            attribution = '1d_Click/View'

            # insights/campaign_name得到 "廣告包名稱"
            if 'campaign_name' in acc_insight:
                campaign_name = acc_insight['campaign_name']
                if '⛔️股市爆料同學會' in campaign_name:
                    campaign_name = campaign_name.replace('⛔️', '')

            # insights/adset_name得到 "廣告set名稱"
            if 'adset_name' in acc_insight:
                adset_name = acc_insight['adset_name']

            # insights/adset_name得到 "廣告名稱"
            if 'ad_name' in acc_insight:
                ad_name = acc_insight['ad_name']

            # insights/cpm得到 "每一次廣告的曝光成本"
            if 'cpm' in acc_insight:
                cpm = acc_insight['cpm']

            # insights/ctr得到 "廣告點擊率"
            if 'ctr' in acc_insight:
                ctr = acc_insight['ctr']

            # insights/ctr得到 "廣告觸及人數"
            if 'impressions' in acc_insight:
                impressions = acc_insight['impressions']

            # insights/spend得到 "總支出成本"
            if 'spend' in acc_insight:    
                amount_spent = acc_insight['spend']

            # insights/objective得到 "廣告目標"
            if 'objective' in acc_insight:
                objective = acc_insight['objective']

            # insights/date_stop得到 "報表終止日期"
            if 'date_stop' in acc_insight:
                reporting_date = datetime.strptime(acc_insight['date_stop'], '%Y-%m-%d').strftime('%Y%m%d')

            # insights/actions得到 "軟體下載數/課程購買數"
            if 'actions' in acc_insight:
                for action in acc_insight['actions']:
                    if action['action_type'] == 'mobile_app_install': 
                        amount_install = action['value']
                    if action['action_type'] == 'purchase':
                        amount_purchase = action['value']

            # insights/action_values得到 "購買轉換價值"
            if 'action_values' in acc_insight:
                for ac_values in acc_insight['action_values']:
                    if ac_values['action_type'] == 'omni_purchase':
                        purchase_conversion_value = ac_values['value']

            # insights/conversions得到 "訂閱數"
            if 'conversions' in acc_insight:
                for cv in acc_insight['conversions']:
                    if cv['action_type'] == 'subscribe_total':
                        try:
                            amount_subscribe = cv['7d_click']
                        except:
                            amount_subscribe = cv['value']

            # insights/conversion_values得到 "訂閱轉換價值"             
            if 'conversion_values' in acc_insight:
                for cv_values in acc_insight['conversion_values']:
                    if cv_values['action_type'] == 'subscribe_total':
                        try:
                            subscribe_conversion_value = cv_values['7d_click']
                        except:
                            subscribe_conversion_value = cv_values['value']

            # 將資料以List的形式儲存
            try:
                Campaign_Name.append(campaign_name)
                Adset_Name.append(adset_name)
                Ad_Name.append(ad_name)
                Objective.append(objective)
                Amount_Spent.append(amount_spent)
                Amount_Install.append(amount_install)
                Amount_Purchase.append(amount_purchase)
                Purchase_Conversion_Value.append(purchase_conversion_value)
                Amount_Subscribe.append(amount_subscribe)
                Subscribe_Conversion_Value.append(subscribe_conversion_value)
                Reporting_Date.append(reporting_date)
                CPM.append(cpm)
                CTR.append(ctr)
                Impressions.append(impressions)
                Act_ID.append(act_id)
                Product_Name.append(product_name)
                Attribution.append(attribution)
            except UnicodeEncodeError as e:
                print(e)
            
#             print(acc_insight['campaign_name'], '\n', acc_insight['adset_name'], '\n', acc_insight['ad_name'], '\n', acc_insight)

        print('第{}次'.format(count)) 
#         time.sleep(30)
            
        # 當成是執行4次時，暫停時間300秒
        if count%12 == 0:
            time.sleep(10)

        # 當開始時間和結束時間相等時，立即結束程式
        if start_time == end_time:
            break
        # 將資料以Dataaframe的方式儲存
        raw_data = {'Campaign_Name': Campaign_Name,
                 'Adset_Name': Adset_Name,
                 'Ad_Name': Ad_Name,
                 'Objective': Objective,
                 'Amount_Spent': Amount_Spent,
                 'Amount_Install': Amount_Install,
                 'Amount_Purchase': Amount_Purchase,
                 'Purchase_Conversion_Value': Purchase_Conversion_Value,
                 'Amount_Subscribe': Amount_Subscribe,
                 'Subscribe_Conversion_Value': Subscribe_Conversion_Value,
                 'CPM': CPM,
                 'CTR': CTR,
                 'Impressions': Impressions,
                 'Reporting_Date': Reporting_Date,
                 'Product_Name': Product_Name,
                 'Act_ID': Act_ID,
                 'Attribution': Attribution,}   
        df = pd.DataFrame.from_dict(raw_data)

    # flag確立program完整結束
    print('--- %s seconds ---' % (time.time() - run_time))
    
    # 如果有蒐集到任何數據，any(raw_data) == True
    if any(raw_data.values()):
        print('有acc_insight資料') 
        insight_flag = 1 
    else:
        print('沒東西')
        insight_flag = 0
    return df, insight_flag

## pickle檔案儲存法

In [45]:
# 確認一下廣告到底有沒有打進去(這要放外面，乾你娘)
act_get_acc_insight = dict() 
pickle_file_name = 'df_acc_of_three'
df = pd.DataFrame()

suspend_sign = 0
try: 
    for product_name, act_id in Act_list.items():
        
        # 起始的時間資料
        start_time = datetime(2019, 12, 26).date()
#         start_time = reporting_date
        end_time = start_time + relativedelta(weeks = 1)
#         obj_time = datetime.now().date()
        obj_time = datetime(2020, 1, 26).date()

        # df will refresh every time, since there are many Ads-accounts to work with
        df, insight_flag = main_prog(act_id, product_name, start_time, end_time, obj_time)
        
        #### 這裡可以寫成一個 function

        # 確認哪些 act_id 是已經有廣告資料
        if insight_flag == 1:
            act_get_acc_insight[act_id] = 1
            # 如果 folder 內，沒有 pickle 資料，則創造一個，並且
            if | not os.path.isfile("{}".format(pickle_file_name)):

                # 將資料儲存成pickle檔案
                df.to_pickle(pickle_file_name)
                # 問題解決了
                print('可能這裡出問題')
#                 suspend_sign += 1
            
            else:
                df_accu = pd.read_pickle(pickle_file_name)
                
                # 合併 df_accu & df，
#                 df_combo = df_accu.append(df)
                df_combo = pd.concat([df_accu, df], ignore_index = True) # ignore index
                # 將資料儲存成 pickle 檔案
                df_combo.to_pickle(pickle_file_name)
#                 suspend_sign += 1

            print(df)
#             print('suspend_sign: ', suspend_sign)

        else: 
            act_get_acc_insight[act_id] = 0
            print(df)
        
        # 程式執行 ? 次就停止
        if suspend_sign == 3:
            break
    sucMsg = 'Working well'
    
except Exception as e:
    error_class = e.__class__.__name__ #取得錯誤類型
    detail = e.args[0] #取得詳細內容
    _, _, tb = sys.exc_info() #取得Call Stack
    lastCallStack = traceback.extract_tb(tb)[-1] #取得Call Stack的最後一筆資料
    fileName = lastCallStack[0] #取得發生的檔案名稱
    lineNum = lastCallStack[1] #取得發生的行號
    funcName = lastCallStack[2] #取得發生的函數名稱
    errMsg = "File \"{}\", line {}, in {}: [{}] {}".format(fileName, lineNum, funcName, error_class, detail)
    print(errMsg, '\n')

SyntaxError: invalid syntax (<ipython-input-45-b4a485f156ad>, line 26)

## DF 儲存法

In [10]:
# 確認一下廣告到底有沒有打進去(這要放外面，乾你娘)
act_get_acc_insight = dict() 
df_origin = pd.DataFrame()
count_for_df_combo = 0

suspend_sign = 0
try: 
    for product_name, act_id in Act_list.items():
        
        # 起始的時間資料
#         start_time = datetime(2020, 1, 26).date()
        start_time = reporting_date + relativedelta(days = 1)
        end_time = start_time + relativedelta(weeks = 1)
#         obj_time = datetime.now().date()
#         obj_time = datetime(2020, 3, 2).date()
        obj_time = start_time + relativedelta(months = 1)
    
        # df will refresh every time, since there are many Ads-accounts to work with
        df, insight_flag = main_prog(act_id, product_name, start_time, end_time, obj_time)
        
        #### 這裡可以寫成一個 function

        # 確認哪些 act_id 是已經有廣告資料
        if insight_flag == 1:
            act_get_acc_insight[act_id] = 1
            
            if df_origin.empty & count_for_df_combo == 0:
            # 合併 df_accu & df，
#                 df_combo = df_accu.append(df)
                df_combo = pd.concat([df_origin, df], ignore_index = True) # ignore index
#                 suspend_sign += 1
                count_for_df_combo += 1
            else: 
                df_combo = pd.concat([df_combo, df], ignore_index = True) # ignore index
#             print('suspend_sign: ', suspend_sign)
            print(df)
    
        else: 
            act_get_acc_insight[act_id] = 0
            print(df)
        
        # 程式執行 ? 次就停止
        if suspend_sign == 3:
            break
    sucMsg = 'Working well'
    
except Exception as e:
    error_class = e.__class__.__name__ #取得錯誤類型
    detail = e.args[0] #取得詳細內容
    _, _, tb = sys.exc_info() #取得Call Stack
    lastCallStack = traceback.extract_tb(tb)[-1] #取得Call Stack的最後一筆資料
    fileName = lastCallStack[0] #取得發生的檔案名稱
    lineNum = lastCallStack[1] #取得發生的行號
    funcName = lastCallStack[2] #取得發生的函數名稱
    errMsg = "File \"{}\", line {}, in {}: [{}] {}".format(fileName, lineNum, funcName, error_class, detail)
    print(errMsg, '\n')

2020-05-19
2020-05-26


c:\users\user1\appdata\local\programs\python\python38\lib\site-packages\facebook_business\utils\api_utils.py:30: UserWarning: value of level might not be compatible.  Expect level_enum;  got <class 'set'>
  warnings.warn(message)
c:\users\user1\appdata\local\programs\python\python38\lib\site-packages\facebook_business\utils\api_utils.py:30: UserWarning: value of action_attribution_windows might not be compatible.  Expect list<action_attribution_windows_enum>;  got <class 'set'>
  warnings.warn(message)


第1次
2020-05-26
2020-06-02
第2次
2020-06-02
2020-06-09
第3次
2020-06-09
2020-06-16
第4次
2020-06-16
2020-06-19
第5次
--- 3.3384485244750977 seconds ---
沒東西
Empty DataFrame
Columns: [Campaign_Name, Adset_Name, Ad_Name, Objective, Amount_Spent, Amount_Install, Amount_Purchase, Purchase_Conversion_Value, Amount_Subscribe, Subscribe_Conversion_Value, CPM, CTR, Impressions, Reporting_Date, Product_Name, Act_ID, Attribution]
Index: []
2020-05-19
2020-05-26


c:\users\user1\appdata\local\programs\python\python38\lib\site-packages\facebook_business\utils\api_utils.py:30: UserWarning: value of level might not be compatible.  Expect level_enum;  got <class 'set'>
  warnings.warn(message)
c:\users\user1\appdata\local\programs\python\python38\lib\site-packages\facebook_business\utils\api_utils.py:30: UserWarning: value of action_attribution_windows might not be compatible.  Expect list<action_attribution_windows_enum>;  got <class 'set'>
  warnings.warn(message)


第1次
2020-05-26
2020-06-02
第2次
2020-06-02
2020-06-09
第3次
2020-06-09
2020-06-16
第4次
2020-06-16
2020-06-19
第5次
--- 284.90060925483704 seconds ---
有acc_insight資料
                        Campaign_Name                       Adset_Name  \
0                     期貨電子盤_下載_IOS(舊)                       興趣期貨受眾_IOS   
1                     期貨電子盤_下載_IOS(舊)                    期貨電子盤類似受眾_IOS   
2              20200923_籌碼K線_最後衝刺_iOS                以股市為興趣_影片_android   
3              20200923_籌碼K線_最後衝刺_iOS                    以股市為興趣_影片_ios   
4              20200923_籌碼K線_最後衝刺_iOS                     以股市為興趣圖片_iOS   
...                               ...                              ...   
5044  20200608_期權先生_購買_首購銷售_圖片影音_大眾受眾  20200616_期權先生_購買_首購銷售_圖片_大眾粉專受眾   
5045  20200608_期權先生_購買_首購銷售_圖片影音_大眾受眾    20200616_期權先生_購買_首購銷售_圖片_波段課程   
5046  20200608_期權先生_購買_首購銷售_圖片影音_大眾受眾    20200616_期權先生_購買_首購銷售_圖片_波段軟體   
5047  20200608_期權先生_購買_首購銷售_圖片影音_大眾受眾    20200616_期權先生_購買_首購銷售_圖片_波段粉專   
5048  20200608_期權先生_購買_首購銷售_

...

第1次
2020-05-26
2020-06-02
第2次
2020-06-02
2020-06-09
第3次
2020-06-09
2020-06-16
第4次
2020-06-16
2020-06-19
第5次
--- 2.4237115383148193 seconds ---
沒東西
Empty DataFrame
Columns: [Campaign_Name, Adset_Name, Ad_Name, Objective, Amount_Spent, Amount_Install, Amount_Purchase, Purchase_Conversion_Value, Amount_Subscribe, Subscribe_Conversion_Value, CPM, CTR, Impressions, Reporting_Date, Product_Name, Act_ID, Attribution]
Index: []
2020-05-19
2020-05-26


c:\users\user1\appdata\local\programs\python\python38\lib\site-packages\facebook_business\utils\api_utils.py:30: UserWarning: value of level might not be compatible.  Expect level_enum;  got <class 'set'>
  warnings.warn(message)
c:\users\user1\appdata\local\programs\python\python38\lib\site-packages\facebook_business\utils\api_utils.py:30: UserWarning: value of action_attribution_windows might not be compatible.  Expect list<action_attribution_windows_enum>;  got <class 'set'>
  warnings.warn(message)


第1次
2020-05-26
2020-06-02
第2次
2020-06-02
2020-06-09
第3次
2020-06-09
2020-06-16
第4次
2020-06-16
2020-06-19
第5次
--- 2.6202661991119385 seconds ---
沒東西
Empty DataFrame
Columns: [Campaign_Name, Adset_Name, Ad_Name, Objective, Amount_Spent, Amount_Install, Amount_Purchase, Purchase_Conversion_Value, Amount_Subscribe, Subscribe_Conversion_Value, CPM, CTR, Impressions, Reporting_Date, Product_Name, Act_ID, Attribution]
Index: []
2020-05-19
2020-05-26


c:\users\user1\appdata\local\programs\python\python38\lib\site-packages\facebook_business\utils\api_utils.py:30: UserWarning: value of level might not be compatible.  Expect level_enum;  got <class 'set'>
  warnings.warn(message)
c:\users\user1\appdata\local\programs\python\python38\lib\site-packages\facebook_business\utils\api_utils.py:30: UserWarning: value of action_attribution_windows might not be compatible.  Expect list<action_attribution_windows_enum>;  got <class 'set'>
  warnings.warn(message)


第1次
2020-05-26
2020-06-02
第2次
2020-06-02
2020-06-09
第3次
2020-06-09
2020-06-16
第4次
2020-06-16
2020-06-19
第5次
--- 3.1957592964172363 seconds ---
沒東西
Empty DataFrame
Columns: [Campaign_Name, Adset_Name, Ad_Name, Objective, Amount_Spent, Amount_Install, Amount_Purchase, Purchase_Conversion_Value, Amount_Subscribe, Subscribe_Conversion_Value, CPM, CTR, Impressions, Reporting_Date, Product_Name, Act_ID, Attribution]
Index: []


In [24]:
df_origin = pd.DataFrame()
if df_origin.empty:
    pass

## pickle檔案讀取

In [26]:
df_test = pd.read_pickle(pickle_file_name)
df_test

,Campaign_Name,Adset_Name,Ad_Name,Objective,Amount_Spent,Amount_Install,Amount_Purchase,Purchase_Conversion_Value,Amount_Subscribe,Subscribe_Conversion_Value,CPM,CTR,Impressions,Reporting_Date,Product_Name,Act_ID,Attribution
0,大股東_應用程式下載_IOS,大股東_股票市場興趣受眾,商店頁文案圖片,APP_INSTALLS,115,9,,,,,122.601279,1.599147,938,20210330,大股東_APP,1864362753722473,1d_Click/View
1,大股東_應用程式下載_IOS,大股東_股票市場興趣受眾,商店頁文案圖片,APP_INSTALLS,194,4,1,3577,,,135.191638,0.905923,1435,20210331,大股東_APP,1864362753722473,1d_Click/View
2,大股東_應用程式下載_IOS,大股東_股票市場興趣受眾,商店頁文案圖片,APP_INSTALLS,81,4,,,,,133.004926,1.149425,609,20210401,大股東_APP,1864362753722473,1d_Click/View
3,價值K線_應用程式安裝_iOS,基本面粉絲頁按讚+基本面類似受眾2%,評價頁_圖片,APP_INSTALLS,191,16,,,,,66.759874,2.446697,2861,20210329,價值K線_APP,294383082255305,1d_Click/View
4,價值K線_應用程式安裝_iOS,基本面粉絲頁按讚+基本面類似受眾2%,評價頁_圖片,APP_INSTALLS,191,16,,,,,66.759874,2.446697,2861,20210329,價值K線_APP,294383082255305,1d_Click/View
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13637,林恩如2021Q1_購買_實戰講座,瀏覽恩如報名頁者(付實+初階*3+all),素材：紫色+金色,CONVERSIONS,251,,2,20887,,,179.413867,8.220157,1399,20210123,籌碼K線(原本測試帳號),132799196821088,1d_Click/View
13638,20210105阿勳APP下載_iOS_軟體,IOS_外部投資興趣 (股票or價值投資),台股高檔(圖),APP_INSTALLS,169,11,,,,,133.175729,2.206462,1269,20210123,籌碼K線(原本測試帳號),132799196821088,1d_Click/View
13639,20210105阿勳APP下載_iOS_軟體,IOS_外部投資興趣 (股票or價值投資),財報複雜(圖),APP_INSTALLS,15,,,,,,141.509434,0,106,20210123,籌碼K線(原本測試帳號),132799196821088,1d_Click/View
13640,算利教官_下載_IOS,算利教官_下載_IOS,算利教官_下載_IOS,APP_INSTALLS,999,117,2,10378,,,148.175616,3.292791,6742,20210123,籌碼K線(原本測試帳號),132799196821088,1d_Click/View


## 一般 DF 讀取

In [127]:
df_combo

,Campaign_Name,Adset_Name,Ad_Name,Objective,Amount_Spent,Amount_Install,Amount_Purchase,Purchase_Conversion_Value,Amount_Subscribe,Subscribe_Conversion_Value,CPM,CTR,Impressions,Reporting_Date,Product_Name,Act_ID,Attribution
0,期貨電子盤_下載_IOS(舊),興趣期貨受眾_IOS,創意廣告,APP_INSTALLS,471,32,,,,,58.794158,1.086007,8011,20200322,籌碼K線(原本測試帳號),132799196821088,1d_Click/View
1,期貨電子盤_下載_IOS(舊),期貨電子盤類似受眾_IOS,創意廣告,APP_INSTALLS,23,3,,,,,44.315992,1.734104,519,20200322,籌碼K線(原本測試帳號),132799196821088,1d_Click/View
2,籌碼K線_ 競價下載_行銷活動花費無限制,理財寶相關受眾_出價上限35元_圖片_KUSO畫面_android,圖片,APP_INSTALLS,308,15,1,1288,,,73.263559,2.307326,4204,20200322,籌碼K線(原本測試帳號),132799196821088,1d_Click/View
3,20200923_籌碼K線_最後衝刺_iOS,籌碼K_以股市為興趣_圖片_ios,圖片,APP_INSTALLS,1659,65,1,8888,,,85.313175,1.105626,19446,20200322,籌碼K線(原本測試帳號),132799196821088,1d_Click/View
4,20200923_籌碼K線_最後衝刺_iOS,籌碼K_以股市為興趣_影片_ios,影片,APP_INSTALLS,588,23,,,1,1990,105.357463,1.182584,5581,20200322,籌碼K線(原本測試帳號),132799196821088,1d_Click/View
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2090,羅子原_翻譯訊息APP_20200323,羅子原_翻譯訊息APP_中年,羅子原_翻譯訊息APP_裝置與漸層背景,LINK_CLICKS,193,,,,,,37.280278,5.427854,5177,20200325,大大PM,509426729992360,1d_Click/View
2091,林孟諭_TourGuideLIve_20200323,林孟諭_TGL_0322,林孟諭_TGL_0322,LINK_CLICKS,3,,,,,,56.603774,0,53,20200325,大大PM,509426729992360,1d_Click/View
2092,陳羿涵_Taskr交友軟體_20200323,陳羿涵_TASKR交友軟體_對交友遊戲有興趣的年輕族群,陳羿涵_TASKR交友APP_image1_文案2,LINK_CLICKS,0,,,,,,0,0,27,20200325,大大PM,509426729992360,1d_Click/View
2093,陳萱寧_Hostrack_20200323,陳萱寧＿Hostrack＿中年人,陳萱寧＿Hostrack,LINK_CLICKS,0,,,,,,0,0,12,20200325,大大PM,509426729992360,1d_Click/View


## 增加 SD 和 CA

In [11]:
# pickle 檔案讀取
# df_test = Expand_df(df_test, Last_SN)

# 一般 DF 讀取
df_combo = Expand_df(df_combo, Last_SN)

In [12]:
# pickle 檔案讀取
# Blank_to_None(df_test)

# 一般 DF 讀取
Blank_to_None(df_combo)

,Serial_Number,Campaign_Name,Adset_Name,Ad_Name,Objective,Amount_Spent,Amount_Install,Amount_Purchase,Purchase_Conversion_Value,Amount_Subscribe,Subscribe_Conversion_Value,CPM,CTR,Impressions,Reporting_Date,Category,Product_Name,Act_ID,Attribution
0,202005190007490,期貨電子盤_下載_IOS(舊),興趣期貨受眾_IOS,創意廣告,APP_INSTALLS,87,10,None,None,None,None,61.22449,1.196341,1421,20200519,FB,籌碼K線(原本測試帳號),132799196821088,1d_Click/View
1,202005190007491,期貨電子盤_下載_IOS(舊),期貨電子盤類似受眾_IOS,創意廣告,APP_INSTALLS,33,4,None,None,None,None,60.43956,1.648352,546,20200519,FB,籌碼K線(原本測試帳號),132799196821088,1d_Click/View
2,202005190007492,20200923_籌碼K線_最後衝刺_iOS,以股市為興趣_影片_android,影片,APP_INSTALLS,234,15,None,None,None,None,66.571835,1.422475,3515,20200519,FB,籌碼K線(原本測試帳號),132799196821088,1d_Click/View
3,202005190007493,20200923_籌碼K線_最後衝刺_iOS,以股市為興趣_影片_ios,影片,APP_INSTALLS,236,17,None,None,None,None,71.580224,1.637853,3297,20200519,FB,籌碼K線(原本測試帳號),132799196821088,1d_Click/View
4,202005190007494,20200923_籌碼K線_最後衝刺_iOS,以股市為興趣圖片_iOS,單一圖片_特色介紹_股市走勢,APP_INSTALLS,468,13,None,None,None,None,93.208524,0.597491,5021,20200519,FB,籌碼K線(原本測試帳號),132799196821088,1d_Click/View
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5044,2020061600012534,20200608_期權先生_購買_首購銷售_圖片影音_大眾受眾,20200616_期權先生_購買_首購銷售_圖片_大眾粉專受眾,20200608_期權先生_購買_首購銷售_圖片_大眾粉專受眾,CONVERSIONS,1,None,None,None,None,None,142.857143,0,7,20200616,FB,籌碼K線(原本測試帳號),132799196821088,1d_Click/View
5045,2020061600012535,20200608_期權先生_購買_首購銷售_圖片影音_大眾受眾,20200616_期權先生_購買_首購銷售_圖片_波段課程,20200616_期權先生_購買_首購銷售_圖片_波段課程,CONVERSIONS,0,None,None,None,None,None,0,0,1,20200616,FB,籌碼K線(原本測試帳號),132799196821088,1d_Click/View
5046,2020061600012536,20200608_期權先生_購買_首購銷售_圖片影音_大眾受眾,20200616_期權先生_購買_首購銷售_圖片_波段軟體,20200616_期權先生_購買_首購銷售_圖片_波段軟體,CONVERSIONS,0,None,None,None,None,None,0,0,5,20200616,FB,籌碼K線(原本測試帳號),132799196821088,1d_Click/View
5047,2020061600012537,20200608_期權先生_購買_首購銷售_圖片影音_大眾受眾,20200616_期權先生_購買_首購銷售_圖片_波段粉專,20200616_期權先生_購買_首購銷售_圖片_波段粉專,CONVERSIONS,0,None,None,None,None,None,0,0,5,20200616,FB,籌碼K線(原本測試帳號),132799196821088,1d_Click/View


In [52]:
# df_test
df_combo 

,Serial_Number,Campaign_Name,Adset_Name,Ad_Name,Objective,Amount_Spent,Amount_Install,Amount_Purchase,Purchase_Conversion_Value,Amount_Subscribe,Subscribe_Conversion_Value,CPM,CTR,Impressions,Reporting_Date,Category,Product_Name,Act_ID,Attribution
0,201912260000,期貨電子盤_下載_IOS(舊),興趣期貨受眾_IOS,創意廣告,APP_INSTALLS,4,None,None,None,None,None,111.111111,2.777778,36,20191226,FB,籌碼K線(原本測試帳號),132799196821088,1d_Click/View
1,201912260001,期貨電子盤_下載_IOS(舊),期貨電子盤類似受眾_IOS,創意廣告,APP_INSTALLS,25,None,None,None,None,None,83.892617,1.342282,298,20191226,FB,籌碼K線(原本測試帳號),132799196821088,1d_Click/View
2,201912260002,籌碼K線_ 競價下載_行銷活動花費無限制,理財寶相關受眾_出價上限35元_圖片_KUSO畫面_android,圖片,APP_INSTALLS,48,1,None,None,None,None,99.378882,0.414079,483,20191226,FB,籌碼K線(原本測試帳號),132799196821088,1d_Click/View
3,201912260003,籌碼K線_ 競價下載_行銷活動花費無限制,理財寶相關受眾_出價上限50元_圖片_App畫面_android,籌碼K線_前5%類似受眾_20180927,APP_INSTALLS,101,4,None,None,None,None,115.034169,1.366743,878,20191226,FB,籌碼K線(原本測試帳號),132799196821088,1d_Click/View
4,201912260004,期貨電子盤_下載_IOS(舊),安裝過籌碼K線APP_IOS,期貨電子盤,APP_INSTALLS,1,None,None,None,None,None,111.111111,0,9,20191226,FB,籌碼K線(原本測試帳號),132799196821088,1d_Click/View
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1008,202001230001008,武財神_點燈活動,過去180天有與理財寶作者互動的人,過去180天有與理財寶作者互動的人,CONVERSIONS,2327,None,7,15460,None,None,74.700652,2.362685,31151,20200123,FB,籌碼K線(原本測試帳號),132799196821088,1d_Click/View
1009,202001230001009,武財神_點燈活動,曾瀏覽武財神LP的人,曾瀏覽武財神LP的人,CONVERSIONS,978,None,6,5994,None,None,83.554037,4.647587,11705,20200123,FB,籌碼K線(原本測試帳號),132799196821088,1d_Click/View
1010,202001230001010,武財神_點燈活動,對理財寶粉專財運文章有興趣的人,對理財寶粉專財運文章有興趣的人,CONVERSIONS,1418,None,9,8991,None,None,158.595235,3.25467,8941,20200123,FB,籌碼K線(原本測試帳號),132799196821088,1d_Click/View
1011,202001230001011,武財神_點燈活動,與理財寶互動的人,與理財寶互動的人,CONVERSIONS,979,None,3,2997,None,None,73.620093,3.248609,13298,20200123,FB,籌碼K線(原本測試帳號),132799196821088,1d_Click/View


In [75]:
df_test4test = df_test.copy()
# df_test4test['Act_ID'] = df_test4test['Act_ID'].astype(int) 
df_test4test['Act_ID'] = pd.to_numeric(df_test4test['Act_ID'])

In [76]:
df_test4test

,Serial_Number,Campaign_Name,Adset_Name,Ad_Name,Objective,Amount_Spent,Amount_Install,Amount_Purchase,Purchase_Conversion_Value,Amount_Subscribe,Subscribe_Conversion_Value,CPM,CTR,Impressions,Reporting_Date,Category,Product_Name,Act_ID,Attribution
0,2021033000053115,大股東_應用程式下載_IOS,大股東_股票市場興趣受眾,商店頁文案圖片,APP_INSTALLS,115,9,None,None,None,None,122.601279,1.599147,938,20210330,FB,大股東_APP,1864362753722473,1d_Click/View
1,2021033100053116,大股東_應用程式下載_IOS,大股東_股票市場興趣受眾,商店頁文案圖片,APP_INSTALLS,194,4,1,3577,None,None,135.191638,0.905923,1435,20210331,FB,大股東_APP,1864362753722473,1d_Click/View
2,2021040100053117,大股東_應用程式下載_IOS,大股東_股票市場興趣受眾,商店頁文案圖片,APP_INSTALLS,81,4,None,None,None,None,133.004926,1.149425,609,20210401,FB,大股東_APP,1864362753722473,1d_Click/View
3,2021032900053118,價值K線_應用程式安裝_iOS,基本面粉絲頁按讚+基本面類似受眾2%,評價頁_圖片,APP_INSTALLS,191,16,None,None,None,None,66.759874,2.446697,2861,20210329,FB,價值K線_APP,294383082255305,1d_Click/View
4,2021032900053119,價值K線_應用程式安裝_iOS,基本面粉絲頁按讚+基本面類似受眾2%,評價頁_圖片,APP_INSTALLS,191,16,None,None,None,None,66.759874,2.446697,2861,20210329,FB,價值K線_APP,294383082255305,1d_Click/View
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340,2021040100053455,Q2_艾蜜莉下載_IOS_自動,20210331_下載_IOS,20210331_IOS_混和素材,APP_INSTALLS,26,3,None,None,1,390,92.198582,2.836879,282,20210401,FB,艾蜜莉_APP,2813564628898257,1d_Click/View
341,2021040200053456,Q2_艾蜜莉下載_IOS_自動,20210331_下載_IOS,20210331_IOS_混和素材,APP_INSTALLS,44,6,None,None,None,None,81.180812,2.767528,542,20210402,FB,艾蜜莉_APP,2813564628898257,1d_Click/View
342,2021040300053457,Q2_艾蜜莉下載_IOS_自動,20210331_下載_IOS,20210331_IOS_混和素材,APP_INSTALLS,39,5,None,None,None,None,73.863636,2.272727,528,20210403,FB,艾蜜莉_APP,2813564628898257,1d_Click/View
343,2021040400053458,Q2_艾蜜莉下載_IOS_自動,20210331_下載_IOS,20210331_IOS_混和素材,APP_INSTALLS,52,6,None,None,None,None,66.752246,1.797176,779,20210404,FB,艾蜜莉_APP,2813564628898257,1d_Click/View


In [77]:
print(df_test4test['Serial_Number'].dtype)
print(df_test4test['Act_ID'].dtype)

int64
int64


## Insert 到 SQL 內部

In [ ]:
# 寫入資料
def InsertSQL(df, TableName):
    str_query = """
        INSERT INTO [CMAPP].[dbo].[{}]([Serial_Number],[Campaign_Name],[Adset_Name],[Ad_Name],[Objective],[Amount_Spent],[Amount_Install],\
        [Amount_Purchase],[Purchase_Conversion_Value],[Amount_Subscribe],[Subscribe_Conversion_Value],[CPM],[CTR],[Impressions],[Reporting_Date]\
        ,[Category],[Product_Name],[Act_ID],[Attribution])VALUES({})
        """
    # 將csv DataFrame每筆資料變成 list 型態
    insert_data = df.values.tolist()
    for i in range(len(insert_data)):
#         print(str_query.format(TableName, str(insert_data[i]).strip('[]').replace('None', 'null')))      
        DBdata_cr.execute(str_query.format(TableName, str(insert_data[i]).strip('[]').replace('None', 'null')))
        conn_DBdata.commit() 

# pickle 檔案
# df_test['Reporting_Date'] = df_test['Reporting_Date'].astype('str')
# df_test4test['Act_ID'] = pd.to_numeric(df_test4test['Act_ID'])
# InsertSQL(df_test4test, TableName)

# 一般 DF 
df_combo['Reporting_Date'] = df_combo['Reporting_Date'].astype('str')
df_combo['Act_ID'] = pd.to_numeric(df_combo['Act_ID'])
InsertSQL(df_combo, TableName)

In [35]:
df_accu = pd.read_pickle(pickle_file_name)
df_accu

,Campaign_Name,Adset_Name,Ad_Name,Objective,Amount_Spent,Amount_Install,Amount_Purchase,Purchase_Conversion_Value,Amount_Subscribe,Subscribe_Conversion_Value,CPM,CTR,Impressions,Reporting_Date,Product_Name,Act_ID,Attribution
0,大股東_應用程式下載_IOS,大股東_股票市場興趣受眾,商店頁文案圖片,APP_INSTALLS,115,9,,,,,122.601279,1.599147,938,20210330,大股東_APP,1864362753722473,1d_Click/View
1,大股東_應用程式下載_IOS,大股東_股票市場興趣受眾,商店頁文案圖片,APP_INSTALLS,194,4,1,3577,,,135.191638,0.905923,1435,20210331,大股東_APP,1864362753722473,1d_Click/View
2,大股東_應用程式下載_IOS,大股東_股票市場興趣受眾,商店頁文案圖片,APP_INSTALLS,81,4,,,,,133.004926,1.149425,609,20210401,大股東_APP,1864362753722473,1d_Click/View
3,價值K線_應用程式安裝_iOS,基本面粉絲頁按讚+基本面類似受眾2%,評價頁_圖片,APP_INSTALLS,191,16,,,,,66.759874,2.446697,2861,20210329,價值K線_APP,294383082255305,1d_Click/View
4,價值K線_應用程式安裝_iOS,基本面粉絲頁按讚+基本面類似受眾2%,評價頁_圖片,APP_INSTALLS,191,16,,,,,66.759874,2.446697,2861,20210329,價值K線_APP,294383082255305,1d_Click/View
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
389,林子揚_流量廣告,林子揚_流量廣告_受眾B版,林子揚_流量廣告_素材A,LINK_CLICKS,97,,,,,,2.469953,0.010185,39272,20210320,大大PM,509426729992360,1d_Click/View
390,鍾凱茵_流量廣告,鍾凱茵_流量廣告_一鍵,鍾凱茵_Landing page_一鍵,LINK_CLICKS,120,,,,,,84.092502,1.962158,1427,20210320,大大PM,509426729992360,1d_Click/View
391,陳霈翎_流量廣告,陳霈翎_流量廣告_31-40台灣旅遊,陳霈翎_流量廣告_31-40台灣旅遊_曝光,LINK_CLICKS,0,,,,,,0,10.526316,19,20210320,大大PM,509426729992360,1d_Click/View
392,陳霈翎_流量廣告,陳霈翎_流量廣告_22-30台灣旅遊 - B,陳霈翎_流量廣告_22-30台灣旅遊_搶先體驗,LINK_CLICKS,60,,,,,,69.284065,3.117783,866,20210320,大大PM,509426729992360,1d_Click/View


""


In [79]:
file_name = 'df_accutest'
import pandas as pd
 
# intialise data of lists.
data = {'Name':['Tom', 'nick', 'krish', 'jack'],
        'Age':[20, 21, 19, 18]}
 
# Create DataFrame
df = pd.DataFrame(data)

df = df.append(df)
# Print the output.
print(df)

    Name  Age
0    Tom   20
1   nick   21
2  krish   19
3   jack   18
0    Tom   20
1   nick   21
2  krish   19
3   jack   18


In [80]:
df.to_pickle(file_name)
df_accu = pd.read_pickle(file_name)
df_combo = pd.concat([df_accu, df], ignore_index = True) # ignore index
df_combo.to_pickle(file_name)

df_combo = pd.read_pickle(file_name)
df_combo

,Name,Age
0,Tom,20
1,nick,21
2,krish,19
3,jack,18
4,Tom,20
5,nick,21
6,krish,19
7,jack,18
8,Tom,20
9,nick,21


In [63]:
# file_name = 'test'
df.to_pickle(file_name)
df_accu = pd.read_pickle(file_name)
df_combo = pd.concat([df_accu, df], ignore_index = True) # ignore index
df_combo.to_pickle(file_name)

df_combo = pd.read_pickle(file_name)
df_combo

,Name,Age
0,Tom,20
1,nick,21
2,krish,19
3,jack,18
4,Tom,20
5,nick,21
6,krish,19
7,jack,18
